# preprocess for all data

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.patheffects as path_effects

In [ ]:
df=pd.read_json('/content/train_data.json')

In [ ]:
df=pd.concat([df,pd.json_normalize(df['targetAudience'])], axis=1)

In [ ]:
df=df.drop(['targetAudience','id'], axis=1)

In [ ]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 2.2 MB/s eta 0:00:00


In [ ]:
import osmnx as ox
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="leaders-of-digital")
def get_lat_lon(city_name):
    location = geolocator.geocode(city_name)
    return location.latitude, location.longitude

# Function to get the OSM polygons
def get_osm_polygons(lat, lon, level):
    gdf = ox.geometries.geometries_from_point((lat, lon), dist=50000, tags={'admin_level': str(level)})
    return gdf

a, b = get_lat_lon("Moscow")
data = get_osm_polygons(a, b, 8)

<ipython-input-4-dc9822d33f67>:11: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries.geometries_from_point((lat, lon), dist=50000, tags={'admin_level': str(level)})


In [ ]:
from shapely import geometry
import pandas as pd
# Районы Москвы в формате район - координаты
raions_moskvi = pd.DataFrame(row for idx, row in data.iterrows() if type(row['geometry']) == Polygon)[['geometry', 'name']]

def delta_points(point):
    p1 = geometry.Point(point.x + 0.001, point.y + 0.001)
    p2 = geometry.Point(point.x + 0.001, point.y - 0.001)
    p3 = geometry.Point(point.x - 0.001, point.y + 0.001)
    p4 = geometry.Point(point.x - 0.001, point.y - 0.001)
    return [p1, p2, p3, p4]

# Функция для определения в каком районе находится точка
def check_point_in_polygons(point, polygons):
    point = geometry.Point(point)
    outp = []
    pts = delta_points(point)
    for p in pts:
        for ind, polygon in enumerate(polygons['geometry']):
            if polygon.contains(p) and polygons['name'][ind] not in outp:
                outp.append(polygons['name'][ind])
    if len(outp) > 0:
        return outp
    return -1

check_point_in_polygons((37.7427, 55.8097), raions_moskvi)

['район Богородское', 'район Преображенское']

In [ ]:
dists=raions_moskvi['name'].to_list()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
mega_df=pd.DataFrame()

In [ ]:
mega_df['district_name']=dists

In [ ]:
mega_df['district_count']=0

In [ ]:
many_tables=[]

In [ ]:
for i in df['points']:
  cords=pd.json_normalize(i)
  cords['district'] = cords.apply(lambda row: check_point_in_polygons((row['lon'],row['lat']), raions_moskvi), axis=1)
  iter_df=mega_df.copy()
  for j in cords['district']:
    if j==-1:
      continue
    for k in j:
      iter_df.loc[iter_df['district_name'] == k, 'district_count'] += 1
  many_tables.append(iter_df)

In [ ]:
for i in range(len(many_tables)):
  many_tables[i]['name']=df['name'][i]
  many_tables[i]['gender']=df['gender'][i]
  many_tables[i]['ageFrom']=df['ageFrom'][i]
  many_tables[i]['ageTo']=df['ageTo'][i]
  many_tables[i]['income']=df['income'][i]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
np.save('save.npy',many_tables)

In [ ]:
down=np.load('save.npy', allow_pickle=True)

# Добавление нескольких фичей - кол-во разных мест (возможно долго, файл с кешем в телеге!) Рамиль сказал 389 записей по хуйне

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.patheffects as path_effects

In [ ]:
down=list(down)

In [ ]:
for i in range(len(down)):
  down[i]=pd.DataFrame(down[i])

In [ ]:
import osmnx as ox
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="leaders-of-digital")
def get_lat_lon(city_name):
    location = geolocator.geocode(city_name)
    return location.latitude, location.longitude

# Function to get the OSM polygons
def get_osm_polygons(lat, lon, level):
    gdf = ox.geometries.geometries_from_point((lat, lon), dist=50000, tags={'admin_level': str(level)})
    return gdf

a, b = get_lat_lon("Moscow")
data = get_osm_polygons(a, b, 8)

<ipython-input-12-dc9822d33f67>:11: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries.geometries_from_point((lat, lon), dist=50000, tags={'admin_level': str(level)})


In [ ]:
from shapely import geometry
from shapely.geometry import Polygon
import pandas as pd

# Районы Москвы в формате район - координаты
raions_moskvi = pd.DataFrame(row for idx, row in data.iterrows() if type(row['geometry']) == Polygon)[['geometry', 'name']]

def delta_points(point):
    p1 = geometry.Point(point.x + 0.001, point.y + 0.001)
    p2 = geometry.Point(point.x + 0.001, point.y - 0.001)
    p3 = geometry.Point(point.x - 0.001, point.y + 0.001)
    p4 = geometry.Point(point.x - 0.001, point.y - 0.001)
    return [p1, p2, p3, p4]

# Функция для определения в каком районе находится точка
def check_point_in_polygons(point, polygons):
    point = geometry.Point(point)
    outp = []
    pts = delta_points(point)
    for p in pts:
        for ind, polygon in enumerate(polygons['geometry']):
            if polygon.contains(p) and polygons['name'][ind] not in outp:
                outp.append(polygons['name'][ind])
    if len(outp) > 0:
        return outp
    return -1

check_point_in_polygons((37.7427, 55.8097), raions_moskvi)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['район Богородское', 'район Преображенское']

In [ ]:
for i in down:
  # i['mall_count']=0
  # i['store_count']=0
  i['shop']=0
  i['leisure']=0
  i['railway']=0
  i['roadst']=0
  i['amenity']=0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from tqdm import tqdm
import pandas as pd
import osmnx as ox
from shapely.geometry import Point, Polygon
tag1 = {
'amenity': ['restaurant', 'cafe', 'hospital', 'school', 'university', 'fast_food', 'bank'],
}
tag2 = {
'shop': True,
}
tag3 = {
'leisure': ['park', 'garden', 'stadium', 'fitness_centre'],
}
tag4 = {
'railway': ['station', 'halt'],
}
tag5 = {
'public_transport': ['stop_position']
}

# Предобработка данных
dots_list = [pd.json_normalize(df['points'][i]) for i in range(len(down))]
# Цикл по точкам
for i in tqdm(range(len(down))):
    dots = dots_list[i]

    # Выполнение запросов к OSM один раз для каждой точки
    results = {}
    for tag_name, tags in [('amenity', tag1), ('shop', tag2), ('leisure', tag3), ('railway', tag4), ('roadst', tag5)]:
        try:
            results[tag_name] = ox.features_from_point((float(dots['lat'][j]), float(dots['lon'][j])), tags, 200)['name'].count().sum()
        except:
            results[tag_name] = 0

    # Цикл по точкам внутри каждой точки
    for j in range(len(dots)):
        dist_with_mag = check_point_in_polygons((dots['lon'][j], dots['lat'][j]), raions_moskvi)
        if dist_with_mag != -1:
            for k in dist_with_mag:
                for tag_name, point_count in results.items():
                    down[i].loc[down[i][0] == k, tag_name] += point_count


100%|██████████| 1547/1547 [1:43:41<00:00,  4.02s/it]


#Добавление одной фичи - кол-во мест

In [ ]:
tags = {
    'amenity': ['cafe', 'university'],
    'shop': ['mall'],
    'railway': ['station'],
}
features = ox.features_from_point((55.7526, 37.6206), tags, 40000)

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
import osmnx as ox
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
for i in tqdm(range(len(down))):
  dots=pd.json_normalize(df['points'][i])
  for j in range(len(dots)):

    polygon = Polygon(delta_points(Point(dots['lon'][j], dots['lat'][j])))
    points = gpd.sjoin(gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon]), features, how='inner')['name'].count().sum()

    dist_with_mag=check_point_in_polygons((dots['lon'][j],dots['lat'][j]),raions_moskvi)
    if points!=0:
      if dist_with_mag!=-1:
        for k in dist_with_mag:
          down[i].loc[down[i][0] == k, 'int_object_count'] +=points

100%|██████████| 1547/1547 [1:52:44<00:00,  4.37s/it]  


In [ ]:
np_down = np.array(down)
np_down.shape

(1547, 147, 8)

In [ ]:
np.save('np_down.npy', np_down)

# Росстат

In [ ]:
tru_data = pd.read_csv('my_data_convert.csv')
ainur_arr = tru_data['0'].values
ilya_arr = down[0][0].values
del_ind = []
for ind, i in enumerate(ainur_arr):
    if i not in ilya_arr:
        del_ind.append(ind) # [64, 67, 85, 145]

In [ ]:
delta_data = tru_data.loc[del_ind]
tru_data.drop(index=del_ind, inplace=True)
delta_data.index = range(142, 146)

In [ ]:
tru_data.sort_values('0', inplace=True)
tru_data.drop(columns= '0', inplace = True)
tru_data.index = range(142)

In [ ]:
# MAIN
new_down = []
delta_data.rename(columns={'0':0}, inplace=True)
for i, data in enumerate(down):
    data.sort_values(0)
    data.drop(index=[132,136,137,138,139], inplace=True) # ошибочные индексы районов Москвы
    data.index = range(142)
    # new_down1[1] = new_down1[1].fillna(value = 0)

    new_down2 = pd.concat([data, tru_data],axis=1)
    new_down1 = pd.concat([new_down2, delta_data])
    new_down1[1] = new_down1[1].fillna(value = 0)
    new_down1.index = range(146)
    new_down1[2] = new_down1[2].fillna(value=new_down1[2][0])
    new_down1[3] = new_down1[3].fillna(new_down1[3][0])
    new_down1[4] = new_down1[4].fillna(new_down1[4][0])
    new_down1[5] = new_down1[5].fillna(new_down1[5][0])
    new_down1[6] = new_down1[6].fillna(new_down1[6][0])
    for i, col in enumerate(new_down1.columns):
        if(i>6):
            new_down1[col] = new_down1[col].fillna(value=new_down1[col].median())
    new_down.append(new_down1)

/tmp/ipykernel_24069/3351626606.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_down1[1] = new_down1[1].fillna(value = 0)
/tmp/ipykernel_24069/3351626606.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_down1[4] = new_down1[4].fillna(new_down1[4][0])
/tmp/ipykernel_24069/3351626606.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcastin

In [ ]:
ss = 0
for i in new_down:
    ss += i.isna().sum()
    i.fillna(value = 0)

In [ ]:
down=#new_down

#Train embeddings

In [ ]:
import numpy as np
import pandas as pd
down=np.load('tmp_17_scaler.npy', allow_pickle=True) # add_rostat   savepoint2_merged_my

In [ ]:
down=list(down)

In [ ]:
for i in range(len(down)):
  down[i]=pd.DataFrame(down[i])

In [ ]:
down[0]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,район Косино-Ухтомский,0,All 25-45 BC,all,25,45,bc,10.49,4.8,348.06,...,10.7,5.9,20.1,7.016893,22.06,172064.6822,36888.0,226026.7442,2,109200
1,район Некрасовка,0,All 25-45 BC,all,25,45,bc,9.55,-3.3,485.14,...,5.7,9.0,18.3,3.593915,47.36,80886.90774,37500.0,224885.2159,2,80400
2,район Новокосино,0,All 25-45 BC,all,25,45,bc,13.37,-0.4,184.29,...,8.3,8.7,14.3,2.538191,63.46,65083.46626,30769.0,144259.9068,1,57300
3,поселение Десёновское,0,All 25-45 BC,all,25,45,bc,10.26,2.1,422.37,...,11.9,9.8,17.8,7.251189,70.93,66785.32233,32000.0,159817.3516,3,88200
4,поселение Клёновское,0,All 25-45 BC,all,25,45,bc,10.94,3.1,1424.73,...,10.3,7.2,21.5,2.898661,24.25,89062.9261,43636.0,247806.0681,10,110100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,район Савёлки,0,All 25-45 BC,all,25,45,bc,12.29,-1.6,173.05,...,7.8,9.4,17.4,9.922977,46.98,67892.40493,31034.0,153535.9949,3,177700
142,район Внуково,0,All 25-45 BC,all,25,45,bc,11.12,8.5,140.27,...,20.1,11.6,18.7,5.38268,14.58,72399.58723,25000.0,97389.78495,3,25000
143,район Восточный,0,All 25-45 BC,all,25,45,bc,10.09,-2.3,111.5,...,6.9,9.2,13.9,5.840016,70.93,60954.21125,21463.0,95454.54545,2,13600
144,район Кунцево,0,All 25-45 BC,all,25,45,bc,12.47,1.0,192.36,...,10.9,9.9,20.6,9.191635,41.32,73257.17633,35000.0,187500.0,8,151000


In [ ]:
cat_features=[2,3,6]
num_features=[1,4,5] + list(range(7,21))

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset

In [ ]:
class SingleDataset(Dataset):
    def __init__(self, x, is_sparse=False):
        self.x = x.astype('float32')
        self.is_sparse = is_sparse

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, index):
        x = self.x[index]
        if self.is_sparse: x = x.toarray().squeeze()
        return x

In [ ]:
import torch
import numpy as np


bce_logits = torch.nn.functional.binary_cross_entropy_with_logits
mse = torch.nn.functional.mse_loss


class TransformerEncoder(torch.nn.Module):
    def __init__(self, embed_dim, num_heads, dropout, feedforward_dim):
        super().__init__()
        self.attn = torch.nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        self.linear_1 = torch.nn.Linear(embed_dim, feedforward_dim)
        self.linear_2 = torch.nn.Linear(feedforward_dim, embed_dim)
        self.layernorm_1 = torch.nn.LayerNorm(embed_dim)
        self.layernorm_2 = torch.nn.LayerNorm(embed_dim)

    def forward(self, x_in):
        attn_out, _ = self.attn(x_in, x_in, x_in)
        x = self.layernorm_1(x_in + attn_out)
        ff_out = self.linear_2(torch.nn.functional.relu(self.linear_1(x)))
        x = self.layernorm_2(x + ff_out)
        return x


class TransformerAutoEncoder(torch.nn.Module):
    def __init__(
            self,
            num_inputs,
            n_cats,
            n_nums,
            hidden_size=1024,
            num_subspaces=8,
            embed_dim=128,
            num_heads=8,
            dropout=0,
            feedforward_dim=512,
            emphasis=.75,
            task_weights=[10, 14],
            mask_loss_weight=2,
        ):
        super().__init__()
        assert hidden_size == embed_dim * num_subspaces
        self.n_cats = n_cats
        self.n_nums = n_nums
        self.num_subspaces = num_subspaces
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.emphasis = emphasis
        self.task_weights = np.array(task_weights) / sum(task_weights)
        self.mask_loss_weight = mask_loss_weight

        self.excite = torch.nn.Linear(in_features=num_inputs, out_features=hidden_size)
        self.encoder_1 = TransformerEncoder(embed_dim, num_heads, dropout, feedforward_dim)
        self.encoder_2 = TransformerEncoder(embed_dim, num_heads, dropout, feedforward_dim)
        self.encoder_3 = TransformerEncoder(embed_dim, num_heads, dropout, feedforward_dim)

        self.mask_predictor = torch.nn.Linear(in_features=hidden_size, out_features=num_inputs)
        self.reconstructor = torch.nn.Linear(in_features=hidden_size + num_inputs, out_features=num_inputs)

    def divide(self, x):
        batch_size = x.shape[0]
        x = x.reshape((batch_size, self.num_subspaces, self.embed_dim)).permute((1, 0, 2))
        return x

    def combine(self, x):
        batch_size = x.shape[1]
        x = x.permute((1, 0, 2)).reshape((batch_size, -1))
        return x

    def forward(self, x):
        x = torch.nn.functional.relu(self.excite(x))

        x = self.divide(x)
        x1 = self.encoder_1(x)
        x2 = self.encoder_2(x1)
        x3 = self.encoder_3(x2)
        x = self.combine(x3)

        predicted_mask = self.mask_predictor(x)
        reconstruction = self.reconstructor(torch.cat([x, predicted_mask], dim=1))
        return (x1, x2, x3), (reconstruction, predicted_mask)

    def split(self, t):
        return torch.split(t, [self.n_cats, self.n_nums], dim=1)

    def feature(self, x):
        attn_outs, _ = self.forward(x)
        return torch.cat([self.combine(x) for x in attn_outs], dim=1)

    def loss(self, x, y, mask, reduction='mean'):
        _, (reconstruction, predicted_mask) = self.forward(x)
        x_cats, x_nums = self.split(reconstruction)
        y_cats, y_nums = self.split(y)
        w_cats, w_nums = self.split(mask * self.emphasis + (1 - mask) * (1 - self.emphasis))

        cat_loss = self.task_weights[0] * torch.mul(w_cats, bce_logits(x_cats, y_cats, reduction='none'))
        num_loss = self.task_weights[1] * torch.mul(w_nums, mse(x_nums, y_nums, reduction='none'))

        reconstruction_loss = torch.cat([cat_loss, num_loss], dim=1) if reduction == 'none' else cat_loss.mean() + num_loss.mean()
        mask_loss = self.mask_loss_weight * bce_logits(predicted_mask, mask, reduction=reduction)

        return reconstruction_loss + mask_loss if reduction == 'mean' else [reconstruction_loss, mask_loss]


class SwapNoiseMasker(object):
    def __init__(self, probas):
        self.probas = torch.from_numpy(np.array(probas))

    def apply(self, X):
        should_swap = torch.bernoulli(self.probas.to(X.device) * torch.ones((X.shape)).to(X.device))
        corrupted_X = torch.where(should_swap == 1, X[torch.randperm(X.shape[0])], X)
        mask = (corrupted_X != X).float()
        return corrupted_X, mask


def test_tf_encoder():
    m = TransformerEncoder(4, 2, .1, 16)
    x = torch.rand((32, 8))
    x = x.reshape((32, 2, 4)).permute((1, 0, 2))
    o = m(x)
    assert o.shape == torch.Size([2, 32, 4])


def test_dae_model():
    m = TransformerAutoEncoder(5, 2, 3, 16, 4, 4, 2, .1, 4, .75)
    x = torch.cat([torch.randint(0, 2, (5, 2)), torch.rand((5, 3))], dim=1)
    f = m.feature(x)
    assert f.shape == torch.Size([5, 16 * 3])
    loss = m.loss(x, x, (x > .2).float())


def test_swap_noise():
    probas = [.2, .5, .8]
    m = SwapNoiseMasker(probas)
    diffs = []
    for i in range(1000):
        x = torch.rand((32, 3))
        noisy_x, _ = m.apply(x)
        diffs.append((x != noisy_x).float().mean(0).unsqueeze(0))

    print('specified : ', probas, ' - actual : ', torch.cat(diffs, 0).mean(0))


if __name__ == '__main__':
    test_tf_encoder()
    test_dae_model()
    test_swap_noise()

specified :  [0.2, 0.5, 0.8]  - actual :  tensor([0.1925, 0.4843, 0.7752])


In [ ]:
df = pd.read_json('train_data.json')

In [ ]:
import torch
import numpy as np
from datetime import datetime
  # from util import AverageMeter
  # from model import SwapNoiseMasker, TransformerAutoEncoder
  # from data import get_data, SingleDataset
from torch.utils.data import DataLoader
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler

np_down = np.array(list(map(lambda x: x.to_numpy(), down)))

encoder = OneHotEncoder(sparse_output = False)
encoder.fit(np_down[:,:,cat_features].reshape(-1, np_down[:,:,cat_features].shape[2]))
scaler=StandardScaler()
scaler.fit(np_down[:,:,num_features].reshape(-1, np_down[:,:,num_features].shape[2]))
X_cats = list()
X_nums = list()
y = list()

n_cats, n_nums = 0, 0

for i in range(len(down)):
  X_num=down[i][num_features].to_numpy()
  X_cat=down[i][cat_features].to_numpy()
  X_cat = encoder.transform(X_cat)
  X_num=scaler.transform(X_num)
  X_cats.append(X_cat)
  X_nums.append(X_num)
  y.append(df['value'][i])
  n_cats=X_cat.shape[1]
  n_nums=X_num.shape[1]

X_nums = np.array(X_nums)
from sklearn.preprocessing import StandardScaler
X_cats = np.array(X_cats)
X = np.hstack((X_cats.reshape(X_cats.shape[0], -1), X_nums.reshape(X_nums.shape[0], -1)))
y = np.array(y)
X.shape

(1547, 13870)

In [ ]:

  # Hyper-params
model_params = dict(
      hidden_size=1024,
      num_subspaces=8,
      embed_dim=128,
      num_heads=8,
      dropout=0.05,
      feedforward_dim=1024,
      emphasis=.75,
      mask_loss_weight=2
  )
batch_size = 786
init_lr = 3e-4
lr_decay = .998
max_epochs = 1000
repeats =  [  2,  2,  3,  200,  200,  350,  400,  508,  2112, 4000,  6093]
probas =  [.95, .4, .7, .9, .9, .9, .9, .9, .9, .9, .25]
swap_probas = sum([[p] * r for p, r in zip(probas, repeats)], [])
#  get data
# X, Y, n_cats, n_nums = get_data()
Y=y

In [ ]:
train_dl = DataLoader(
    dataset=SingleDataset(X),
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)
# setup model
model = TransformerAutoEncoder(
    num_inputs=X.shape[1],
    n_cats=n_cats*146,
    n_nums=n_nums*146,
    **model_params
).cuda()
model_checkpoint = 'model_checkpoint.pth'
print(model)
noise_maker = SwapNoiseMasker(swap_probas)
optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=lr_decay)
for epoch in range(max_epochs):
    t0 = datetime.now()
    model.train()
    meter = AverageMeter()
    for i, x in enumerate(train_dl):
        x = x.cuda()
        x_corrputed, mask = noise_maker.apply(x)
        scheduler.step()
        optimizer.zero_grad()
        loss = model.loss(x_corrputed, x, mask)
        loss.backward()
        optimizer.step()
        meter.update(loss.detach().cpu().numpy())
    delta = (datetime.now() - t0).seconds
    print('\r epoch {:5d} - loss {:.6f} - {:4.6f} sec per epoch'.format(epoch, meter.avg, delta), end='')
torch.save({
        "optimizer": optimizer.state_dict(),
        "scheduler": scheduler.state_dict(),
        "model": model.state_dict()
    }, model_checkpoint
)

TransformerAutoEncoder(
  (excite): Linear(in_features=13870, out_features=1024, bias=True)
  (encoder_1): TransformerEncoder(
    (attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
    (linear_1): Linear(in_features=128, out_features=1024, bias=True)
    (linear_2): Linear(in_features=1024, out_features=128, bias=True)
    (layernorm_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (layernorm_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (encoder_2): TransformerEncoder(
    (attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
    (linear_1): Linear(in_features=128, out_features=1024, bias=True)
    (linear_2): Linear(in_features=1024, out_features=128, bias=True)
    (layernorm_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (layernorm_2): LayerNorm((128,), eps=1e-05, elementwise_affine=

/home/hopper/miniconda3/envs/forML/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


 epoch   999 - loss 0.100831 - 0.000000 sec per epoch

In [ ]:
# extract features
dl = DataLoader(dataset=SingleDataset(X), batch_size=1024, shuffle=False, pin_memory=True, drop_last=False)
features = []
model.eval()
with torch.no_grad():
    for x in dl:
        features.append(model.feature(x.cuda()).detach().cpu().numpy())
features = np.vstack(features)
# downstream supervised regressor
alpha = 1250 # 1000
X = features[:300_000, :]
scores = []
for i in KFold().split(X, Y):
    print(i)
    break
# for train_idx, valid_idx in KFold().split(X, Y):
#     scores.append(mean_squared_error(Y[valid_idx], Ridge(alpha=1250).fit(X[train_idx], Y[train_idx]).predict(X[valid_idx]), squared=False))
print(np.mean(scores))
np.save('../xyz___1.npy', features)

(array([ 310,  311,  312, ..., 1544, 1545, 1546]), array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 1

/home/hopper/miniconda3/envs/forML/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hopper/miniconda3/envs/forML/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


#Eval embeddings

In [ ]:
transformer = TransformerAutoEncoder(num_inputs=X.shape[1],
  n_cats=n_cats*147,
  n_nums=n_nums*147,
  ).cuda()
transformer.load_state_dict(torch.load('model_checkpoint.pth')['model'])
transformer.eval()
dl = DataLoader(dataset=architecture.SingleDataset(X), batch_size=1, shuffle=False, pin_memory=True, drop_last=False)
features = []
with torch.no_grad():
    for x in dl:
        features.append(transformer.feature(x.cuda()).detach().cpu().numpy())
features = np.vstack(features)

#catboost

In [2]:
%pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.1 MB/s eta 0:00:00


In [3]:
from catboost import CatBoostRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
df=pd.read_json('train_data.json')

In [4]:
embeds=np.load('xyz.npy')

In [6]:
y=df['value']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(embeds, y, test_size=0.2, random_state=42)

#Train

In [8]:
model = CatBoostRegressor(iterations=100,
                          depth=6,
                          learning_rate=0.05,
                          loss_function='RMSE')

model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)

0:	learn: 23.6171216	test: 21.6935091	best: 21.6935091 (0)	total: 2.56s	remaining: 4m 13s
99:	learn: 10.8055616	test: 11.1714704	best: 11.1714704 (99)	total: 1m 21s	remaining: 0us

bestTest = 11.17147036
bestIteration = 99



In [9]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [10]:
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'RMSE: {rmse:.4f}')
print(f'R²: {r2:.4f}')
print(f'MAE: {mae:.4f}')

RMSE: 11.1715
R²: 0.7451
MAE: 8.3802


In [ ]:
def show_predictions(y_test,y_pred):
  for yt, yp in zip(y_test,y_pred):
    print(f'{yt} | {yp}')

In [ ]:
slice_idx=10

In [ ]:
y_pred = np.round(y_pred, 2)
show_predictions(y_test[:slice_idx],y_pred[:slice_idx])

2.11 | 5.14
33.41 | 17.09
0.0 | 6.16
0.17 | 17.18
53.04 | 62.81
14.13 | 18.66
2.71 | 2.75
17.21 | 17.48
34.89 | 47.47
19.85 | 14.91


# Addition random forest (хотя вряд-ли)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

In [ ]:
model = RandomForestRegressor(n_estimators=100, max_depth=6, criterion='squared_error')

model.fit(X_train, y_train)

0:	learn: 23.6171216	test: 21.6935091	best: 21.6935091 (0)	total: 2s	remaining: 3m 17s
99:	learn: 10.8055616	test: 11.1714704	best: 11.1714704 (99)	total: 1m 28s	remaining: 0us

bestTest = 11.17147036
bestIteration = 99



In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'RMSE: {rmse:.4f}')
print(f'R²: {r2:.4f}')
print(f'MAE: {mae:.4f}')

RMSE: 11.1715
R²: 0.7451
MAE: 8.3802


# predictions, submission

In [ ]:
predictions=model.predict(features)

In [19]:
submit=pd.DataFrame()

In [25]:
submit['hash']=df['hash']
submit['value']=predictions

In [27]:
submit.to_csv('submission.csv',index=False)